<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Example---Cinema-Simulation" data-toc-modified-id="Example---Cinema-Simulation-1">Example - Cinema Simulation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Define-the-problem" data-toc-modified-id="Define-the-problem-1.0.1">Define the problem</a></span></li><li><span><a href="#Brainstorm-the-algorithm" data-toc-modified-id="Brainstorm-the-algorithm-1.0.2">Brainstorm the algorithm</a></span></li><li><span><a href="#Define-libraries" data-toc-modified-id="Define-libraries-1.0.3">Define libraries</a></span></li><li><span><a href="#Code:-class-definition" data-toc-modified-id="Code:-class-definition-1.0.4">Code: class definition</a></span></li><li><span><a href="#Define-the-function" data-toc-modified-id="Define-the-function-1.0.5">Define the function</a></span></li></ul></li></ul></li></ul></div>

# Example - Cinema Simulation
Simulating Real-World Processes in Python with SimPy
Work through another example of using SimPy from [realpython.com](https://realpython.com/simpy-simulating-with-python/)

### Define the problem
The first step of running a simulation is to choose to process a model. 
In this example, we will imagine we are consulting for a small cinema chain, who have bad reviews due to long waiting times. The company has done some research, and found out that the average customer is willing to wait for at most **10 minutes** between arriving at the venue, and wanting to be sat down. 

Therefore, the problem has been formulated as helping to **get wait times below 10 minutes**.


### Brainstorm the algorithm
Before approaching the problem from a coding perspective, first work out how the process will work in real life. This will ensure that the code is an accurate reflection of what happens in real life. First, list the possible steps someone who visits the cinema would face.

Steps on entering a cinema:
1. **Arrive** at venue
2. **Queue** to buy ticket
3. **Buy** ticket
4. **Queue** to get ticket checked 
5. **Get** ticket checked
6. Decided whether to get drinks/food:
    - If yes, **purchase drinks/food**
    - If no, go to the last step
7. **Go** directly to the seat

Now we have defined the steps above, we can see which parts of the process can be controlled by the cinema chain itself. An example would be how long a customer waits before buying their ticket or drinks/food, and this can be controlled by the number of staff serving these customers.

There are parts of the process that cannot be controlled, such as when the customers are arriving at the venue, or in what volume they will arrive. Since we cannot accurately guess this number, this parameter will have to be filled with available data to determine an appropriate arrival time.


### Define libraries

In [5]:
import random
import statistics

import simpy
print(simpy.__version__)

4.0.1


The goal is to find the optimal number of employees giving an average wait time of **less than 10 minutes**. To define and solve this problem, we will collect a list of waiting times for each customer, from when they enter the venue to when they sit down.

In [7]:
waiting_times = []

### Code: class definition

Build the blueprint for the system, the environment in which the events will happen, such as people moving from one place to another. The environment is the name of the class.

In [8]:
class Cinema(object):
    def __init__(self, env):
        self.env = env

Consider what might be in the Cinema to add to the simulation. As outlined in the steps above, there will be:
    - staff to sell tickets/refreshments (drinks/food)
    - staff can sell the above items
    
Therefore, from the cinema's perspective, the staff are a **resource** who assist the customers in **purchasing items**.
Therefore, we frame the problem as how does the waiting time change depending on the number of customers in each simulation?

So, the next variable to declare in the class is the `num_staff`, which is vital to the results of waiting time.


In [9]:
class Cinema(object):
    def __init__(self, env, num_staff):
        self.env = env
        self.staff = simpy.Resource(env, num_staff)

We know that purchasing a ticket is going to take a certain amount of time, so either use historical data for this, or provide an estimate for this process time. This time can be a range, since the size of the party could be different. In this example we will estimate that it takes between 1 and 3 minutes to buy a ticket.

We will use the `timeout` method from SimPy to mimic this behaviour.

In [10]:
class Cinema(object):
    def __init__(self, env, num_staff):
        self.env = env
        self.staff = simpy.Resource(env, num_staff)
        
    # customer must be passed as a parameter, since they cause the event to occur.
    def purchase_ticket(self, customer):
        yield self.env.timeout(random.randint(1,3))

Declare two more resources:
    - Staff to check tickets
    - Staff to serve food/drinks
These two tasks take a different amount of time, so as before either use historical data, or provide a best guess.

In [14]:
class Cinema(object):
    def __init__(self, env, num_staff, num_checkers, num_servers):
        self.env = env
        self.staff = simpy.Resource(env, num_staff)
        # ticket checker
        self.checker = simpy.Resource(env, num_checkers)
        # food/drinks server
        self.server = simply.Resource(env, num_servers)
        
    # customer must be passed as a parameter, since they cause the event to occur.
    def purchase_ticket(self, customer):
        # process of a customer buying a ticket
        yield self.env.timeout(random.randint(1, 3))
        
    def check_ticket(self, customer):
        # process of a member of staff checking a ticket 
        # this is defined as 3 seconds, don't need a random number
        yield self.env.timeout(3/60) 
        
    def sell_food(self, customer):
        # process of staff selling food
        yield self.env.timeout(random.randint(1, 5))

### Define the function
The environment has been setup by the class above, with the resources and processes defined. All that is left is for a customer to enter the process.

In the process terms, they will:
- arrive at the venue
- request a resource
- wait for the process to complete
- leave

Create a function to simulate this process

In [15]:
def go_to_cinema(env, customer, cinema):
    # customer will be controlled by the environment, so passed into first param
    # varaible customer tracks each person moving through the system
    # final parameter allows us to access the processes defined in the Cinema class
    # define the arrival time as a store to see when the customers arrive
    arrival_time = env.now

Each of the processes from the Cinema should have corresponding requests in `go_to_cinema()`.
The first process in the class is `purchase_ticket()`, using a `staff` resource.

Below is a summary of the processes in the `cinema`, and the request made in the `go_to_cinema` method.

| Process in cinema        | Request in `go_to_cinema()`|
| ------------- |:-------------:| 
| `purchase_ticket()`      | request a member of `staff` | 
| `check_ticket()`      | request a `checker`| 
| `sell_food()` | request a `server`| 

A member of `staff` is a shared resource in the process, so a customer can use the same member of staff, but this member of staff can only help one customer at a time. This needs to be accounted for.

In [18]:
def go_to_cinema(env, customer, cinema):
    # customer will be controlled by the environment, so passed into first param
    # varaible customer tracks each person moving through the system
    # final parameter allows us to access the processes defined in the Cinema class
    # define the arrival time as a store to see when the customers arrive
    arrival_time = env.now
    
    with cinema.staff.request() as request:
        yield request
        yield env.process(cinema.purchase_ticket(customer))

For the above, we see:
- `cinema.staff.request()`: the customer causes a request to call a member of staff, using a `staff` resource
- `yield request`: customer waits for a `staff` to become available if all are currently in use
- `yield env.process()`: the customer uses an available member of `staff` to complete the given process, in this case to purchase the ticket using the class method `cinema.purchase_ticket()`.

Once the member of staff is then freed up, the `customer` will spend time buying their ticket. 
`env.process()` tells the simulation to go to the `Cinema` instance and run the `purchase_ticket()` process on the `customer`.
The customer will repeat the **request, use, release** cycle to get their ticket checked.

In [20]:
def go_to_cinema(env, customer, cinema):
    # customer will be controlled by the environment, so passed into first param
    # varaible customer tracks each person moving through the system
    # final parameter allows us to access the processes defined in the Cinema class
    # define the arrival time as a store to see when the customers arrive
    arrival_time = env.now
    
    with cinema.staff.request() as request:
        yield request
        yield env.process(cinema.purchase_ticket(customer))
        
    with cinema.checker.request() as request:
        yield request
        yield env.process(cinema.check_ticket(customer))

The next part is to add the optional step of buying food/drinks, which is quite random, and we can add the randomness to the function